In [1]:
import os
import numpy as np
import muspy
from tqdm import tqdm
import config
import utils
import tarfile
import shutil
import collections
from matplotlib import pyplot as plt
import json
import tensorflow as tf
import pickle
import json
import pprint
import subprocess

import pandas as pd

from os.path import exists

config_string = "single_instruments_type"
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

conf = config.Config(config_string, ROOT_PATH)

/Users/marcelloceresini/miniconda/envs/env-MusicGeneration/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# sounfont_path downloaded from https://www.dropbox.com/s/dl/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip
# from the download section of http://schristiancollins.com/generaluser.php
# fluidsynth instead is installed by "brew install fluidsynth" (for mac)

soundfont_path = os.path.join(ROOT_PATH, "data", "GeneralUser GS 1.471", "GeneralUser GS v1.471.sf2")


In [3]:
repr_dir = os.path.join(conf.DATA_PATH, "generated_songs", 'repr')
midi_dir = os.path.join(conf.DATA_PATH, "generated_songs", 'midi')
wav_dir  = os.path.join(conf.DATA_PATH, "generated_songs", 'wav')

os.makedirs(repr_dir, exist_ok=True)
os.makedirs(midi_dir, exist_ok=True)
os.makedirs(wav_dir,  exist_ok=True)

In [4]:
utils.tempo_inverse_map(40, conf)

161.26989438654405

In [5]:
for song_path in os.listdir(repr_dir):
    # Load the song file
    songs = np.load(os.path.join(repr_dir, song_path), allow_pickle=True)
    song_filename = os.path.splitext(song_path)[0]
    # Songs are batched: select them one by one
    for i, song in enumerate(songs):

        # for note in song:
        #     print("{:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f} {:3.0f}".format(
        #         note[0],
        #         note[1],
        #         note[2],
        #         note[3],
        #         note[4],
        #         note[5],
        #         note[6],
        #         note[7],
        #         note[8],
        #         note[9],
        #         note[10],
        #     ))

        # df_song = pd.DataFrame(song)       
        #  
        print("---------", song_path, i)
        
        # Use anti-transfomation to convert the song to a muspy object
        converted_song  = utils.anti_tranform_representation(song, conf)   

        path_song_midi  = os.path.join(midi_dir, f'{song_filename}_{i}.mid')
        # Write midi song
        muspy.outputs.write_midi(
            path=path_song_midi,
            music=converted_song,
            backend="pretty_midi",
        )
        # Write to wav
        path_song_wav = os.path.join(wav_dir, f'{song_filename}_{i}.wav')

        command = ["fluidsynth", "-F", path_song_wav, soundfont_path, path_song_midi]
        # command = ["fluidsynth", 'param="synth.parallel-render', 'setint="0"', "-F", path_song_wav, soundfont_path, path_song_midi]
        subprocess.run(command)
        
        # To 
        # column = 3
        # print(df_song[column].value_counts().sort_index())


--------- songs_r&b.npy 0
FluidSynth runtime version 2.3.2
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/Users/marcelloceresini/github/MusicGeneration/data/generated_songs/wav/songs_r&b_0.wav'..
--------- songs_r&b.npy 1
FluidSynth runtime version 2.3.2
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/Users/marcelloceresini/github/MusicGeneration/data/generated_songs/wav/songs_r&b_1.wav'..
--------- songs_r&b.npy 2
FluidSynth runtime version 2.3.2
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file '/Users/marcelloceresini/github/MusicGeneration/data/generated_songs/wav/songs_r&b_2.wav'..
--------- son

In [6]:
# column = 4
# print(df_song[column].value_counts().sort_index())

# print(df_song[column].mean())
# print(df_song[column].min())
# print(df_song[column].max())
# print(df_song[column].std())

NameError: name 'df_song' is not defined